![terrainbento logo](../images/terrainbento_logo.png)


# terrainbento model Basic steady-state solution

This model shows example usage of the Basic model from the TerrainBento package.

This uses has stream power and linear diffusion and has the following governing equation:

$\frac{\partial \eta}{\partial t} = - KQ^{1/2}S + D\nabla^2 \eta$

where $K$ and $D$ are constants, $Q$ is discharge, $S$ is local slope, and $\eta$ is the topography.

Refer to [Barnhart et al. (2019)](https://www.geosci-model-dev.net/12/1267/2019/) for further explaination. For detailed information about creating a Basic model, see [the detailed documentation](https://terrainbento.readthedocs.io/en/latest/source/terrainbento.derived_models.model_basic.html).

This notebook (a) shows the initialization and running of this model, (b) saves a NetCDF file of the topography, which we will use to make an oblique Paraview image of the landscape, and (c) creates a slope-area plot at steady state.

In [ ]:
# import required modules
import os
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["font.size"] = 20
matplotlib.rcParams["pdf.fonttype"] = 42

%matplotlib inline

from landlab import imshow_grid
from landlab.io.netcdf import write_netcdf

from terrainbento import Basic

np.random.seed(42)

In [ ]:
# create the parameter dictionary needed to instantiate the model

U = 0.001
K = 0.001
m = 0.5
n = 1.0

params = {
    # create the Clock.
    "clock": {
        "start": 0,
        "step": 10,
        "stop": 1e7
    },

    # Create the Grid
    "grid": {
        "RasterModelGrid": [
            (25, 40),
            {
                "xy_spacing": 40
            },
            {
                "fields": {
                    "node": {
                        "topographic__elevation": {
                            "random": [{
                                "where": "CORE_NODE"
                            }]
                        }
                    }
                }
            },
        ]
    },

    # Set up Boundary Handlers
    "boundary_handlers": {
        "NotCoreNodeBaselevelHandler": {
            "modify_core_nodes": True,
            "lowering_rate": -U
        }
    },
    # Parameters that control output.
    "output_interval": 1e4,
    "save_first_timestep": True,
    "output_prefix": "basic.",
    "fields": ["topographic__elevation"],

    # Parameters that control process and rates.
    "water_erodibility": K,
    "m_sp": m,
    "n_sp": n,
    "regolith_transport_parameter": 1,
}

In [ ]:
# the tolerance here is high, so that this can run on binder and for tests. (recommended value = 0.001 or lower).
tolerance =.01

In [ ]:
# we can use an output writer to run until the model reaches steady state.
class run_to_steady(object):
    def __init__(self, model):
        self.model = model
        self.last_z = self.model.z.copy()
        self.tolerance = tolerance

    def run_one_step(self):
        if model.model_time > 0:
            diff = (self.model.z[model.grid.core_nodes] -
                    self.last_z[model.grid.core_nodes])
            if max(abs(diff)) <= self.tolerance:
                self.model.clock.stop = model._model_time
                print("Model reached steady state in " +
                      str(model._model_time) + " time units\n")
            else:
                self.last_z = self.model.z.copy()
                if model._model_time <= self.model.clock.stop - self.model.output_interval:
                    self.model.clock.stop += self.model.output_interval

In [ ]:
# initialize the model using the Model.from_dict() constructor.
# We also pass the output writer here.
model = Basic.from_dict(params, output_writers={"class": [run_to_steady]})

# to run the model as specified, we execute the following line:
model.run()

In [ ]:
# MAKE SLOPE-AREA PLOT

# plot nodes that are not on the boundary or adjacent to it
core_not_boundary = np.array(
    model.grid.node_has_boundary_neighbor(model.grid.core_nodes)) == False
plotting_nodes = model.grid.core_nodes[core_not_boundary]

# assign area_array and slope_array
area_array = model.grid.at_node["drainage_area"][plotting_nodes]
slope_array = model.grid.at_node["topographic__steepest_slope"][plotting_nodes]

# calculate predicted values:
truth_area = np.sort(np.unique(area_array))
truth_slope = (U / (K * truth_area**m))**(1. / n)

# instantiate figure and plot
fig = plt.figure(figsize=(6, 3.75))
slope_area = plt.subplot()

# plot the data
slope_area.scatter(area_array,
                   slope_array,
                   marker="o",
                   c="k",
                   label="Model Basic")
slope_area.plot(truth_area,
                truth_slope,
                label="Stream Power Only Analytical Predictions",
                c="gray")

# make axes log and set limits
slope_area.set_xscale("log")
slope_area.set_yscale("log")
slope_area.set_xlim(9 * 10**1, 3 * 10**5)
slope_area.set_ylim(1e-3, 1e-1)

# set x and y labels
slope_area.set_xlabel(r"Drainage area [m$^2$]")
slope_area.set_ylabel("Channel slope [-]")

slope_area.legend(scatterpoints=1, prop={"size": 12})
slope_area.tick_params(axis="x", which="major", pad=7)

plt.show()

In [ ]:
# Save stack of all netcdfs for Paraview to use.
# model.save_to_xarray_dataset(filename="basic.nc",
#                              time_unit="years",
#                              reference_time="model start",
#                              space_unit="meters")

# remove temporary netcdfs
model.remove_output_netcdfs()

In [ ]:
# make a plot of the final steady state topography
imshow_grid(model.grid, "topographic__elevation")

## Challenge
This is not quite a straight line yet. Do you have any idea why? Try to perform a simulation matching the analytical solution. 

## Next Steps

- We recommend you review the [terrainbento manuscript](https://www.geosci-model-dev.net/12/1267/2019/).

- There are three additional introductory tutorials: 

    1) [Introduction terrainbento](../example_usage/Introduction_to_terrainbento.ipynb) 
    
    2) [Introduction to boundary conditions in terrainbento](../example_usage/introduction_to_boundary_conditions.ipynb)
    
    3) [Introduction to output writers in terrainbento](../example_usage/introduction_to_output_writers.ipynb). 
    
    
- Five examples of steady state behavior in coupled process models can be found in the following notebooks:

    1) **This Notebook**: [Basic](model_basic_steady_solution.ipynb) the simplest landscape evolution model in the terrainbento package.

    2) [BasicVm](model_basic_var_m_steady_solution.ipynb) which permits the drainage area exponent to change

    3) [BasicCh](model_basicCh_steady_solution.ipynb) which uses a non-linear hillslope erosion and transport law

    4) [BasicVs](model_basicVs_steady_solution.ipynb) which uses variable source area hydrology

    5) [BasisRt](model_basicRt_steady_solution.ipynb) which allows for two lithologies with different K values